In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import csv

In [34]:
# chromedriver는 다운로드 후 경로 지정을 해줘야 한다. (현재는 같은 폴더 
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [35]:
# 로그인 전용 화면
driver.get('https://nid.naver.com/nidlogin.login')
# 아이디와 비밀번호 입력
time.sleep(2)
driver.find_element_by_name('id').send_keys('tjdcks9243')
driver.find_element_by_name('pw').send_keys('**********')
# 로그인 버튼 클릭
time.sleep(2)
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

In [36]:
k_base_url = 'https://m.cafe.naver.com/ArticleList.nhn?search.clubid=15754634&search.menuid=33'
n_base_url = 'https://m.cafe.naver.com/ArticleList.nhn?search.clubid=15754634&search.menuid=24'
test_list = []

# 국내봉사
driver.get(k_base_url)
# iframe으로 프레임 전환
#     driver.switch_to_frame('cafe_main')
for i in range(0,20):
    time.sleep(0.5)
    driver.find_element_by_xpath('//*[@id="btnNextList"]/a').click()
# # href 속성을 찾아 url을 리스트로 저장한다.
article_list = driver.find_elements_by_css_selector('li.board_box a.txt_area')
article_urls = [ i.get_attribute('href') for i in article_list ]

for link in article_urls:
    test_list.append(link)
    
# # 해외봉사
driver.get(n_base_url)
# iframe으로 프레임 전환
#     driver.switch_to_frame('cafe_main')
for i in range(0,20):
    time.sleep(0.5)
    driver.find_element_by_xpath('//*[@id="btnNextList"]/a').click()
# # href 속성을 찾아 url을 리스트로 저장한다.
article_list = driver.find_elements_by_css_selector('li.board_box a.txt_area')
article_urls = [ i.get_attribute('href') for i in article_list ]

for link in article_urls:
    test_list.append(link)
    
# find_content(test_list)

In [37]:
len(test_list)
# 1000

800

In [38]:
total_list=['제목','작성 날짜','주최사','활동내용','모집 기간','모집 인원','모집 대상',
            '참가 비용 유/무','우대 사항','활동 지역','활동 기간']

f = open('mobile_spec_up4.csv', 'w', encoding='ansi', newline='')
wr = csv.writer(f)
wr.writerow([total_list[0], total_list[1], total_list[2],total_list[3],total_list[4],total_list[5],
             total_list[6],total_list[7],total_list[8],total_list[9],total_list[10]])
f.close()

In [42]:
find_content(test_list)

In [41]:
def find_content(url_list):
    total_lists = []
    for i in range(len(url_list)):
        driver.get(url_list[i])
        html = BeautifulSoup(driver.page_source , 'html.parser')
        content_tags = html.select('#postContent')
        # 본문 내용을 줄바꿈 기준으로 다 붙이기
        # 각 항목마다 쓸데없이 \xa0▷ 요게 붙여져서 정규화 처리
        content = ' '.join([tags.get_text().replace("\xa0","").replace("▷","") for tags in content_tags])
        # 제목가져오기
        titles = html.select("h2.tit")
        # 날짜가져오기
        dates = html.select("span.date.font_l")
        # 전체 개수를 알아보기 위해 t 변수 저장
        # find_location : 크롤링한 content를 처리해주기 위해서 만듬
        t = find_location('mobile_spec_up4.csv',titles, dates, content)
        total_lists.append(t)

In [10]:
def find_location(name,title,dates,test):
    
    fl=[]
    temp_list = []
    new_list= []
    
    
    ############################################
    #
    # 여기에 조건을 주자.
    #
    ############################################

    # 주최사 / 활동명 -> activity_name + 9
    activity_name = test.find("주최사")
    # 활동내용 -> activity_content + 4
    activity_content = test.find("활동내용")
    # 모집 기간 -> recruit_date + 5
    recruit_date = test.find("모집 기간")
    # 모집 인원 + 5
    recruit_people = test.find("모집 인원")
    # 모집 대상 + 5 & 16
    recruit_target = test.find("모집 대상")

    if test.find('(+졸업생 가능여부)') == -1:
        recruit_target_num = 5    
    else:
        recruit_target_num = 16
    
    # 참가 비용 유/무 + 9 
    fee = test.find("참가 비용 유/무")
    # 우대 사항 + 5
    preferential_treatment = test.find("우대 사항")
    # 활동 지역 + 5
    activity_nation = test.find("활동 지역")
    # 활동 기간 + 5
    activity_period = test.find("활동 기간")
    # 활동 혜택 + 5
    activity_bonus = test.find("활동 혜택")
        
#     if (activity_name | activity_content | recruit_date | recruit_people | recruit_target | fee | preferential_treatment | activity_nation | activity_period | activity_bonus | offline | method_appliance | cafe_blog_sns | etc) != -1:
    
    #저장된 제목 값이 없으면, 저장 안 함
    if len(title) != 0:
        # 제목 처리
        titles = title[0].get_text()
        titles=titles.replace("[대외활동]","")
        dates = dates[0].get_text()
        dates_ = re.sub(r"[.]","",dates)
        dates = dates_[:8]
        # find의 값이 없으면, -1을 리턴한다.
        # 활동 이름이 없으면,
        if activity_name != -1:
            fl.append(activity_name)
            fl.append(activity_content)
            fl.append(recruit_date)
            fl.append(recruit_people)
            fl.append(recruit_target)
            fl.append(fee)
            
            fl.append(preferential_treatment)
            fl.append(activity_nation)
            fl.append(activity_period)
            fl.append(activity_bonus)

            a0 = re.sub(r"[-]","",test[fl[0]+9:fl[1]].strip())
            a1 = re.sub(r"[-]","",test[fl[1]+4:fl[2]].strip())
            a2 = re.sub(r"[-]","",test[fl[2]+5:fl[3]].strip())
            a3 = re.sub(r"[-]","",test[fl[3]+5:fl[4]].strip())
            a4 = re.sub(r"[-]","",test[fl[4]+recruit_target_num:fl[5]].strip())

            # 참가 비용이 없다면 모집 대상 <-> 우대사항으로 끊어서 결정해
            if fee == -1:
                a4 = re.sub(r"[-]","",test[fl[4]+recruit_target_num:fl[6]].strip())
                a6 = re.sub(r"[-]","",test[fl[6]+5:fl[7]].strip())
                # 참가비용ㄴ 우대 사항ㄴ, 모집 대상 <-> 활동 지역으로 끊어서 저장
                if preferential_treatment == -1:
                    a4 = re.sub(r"[-]","",test[fl[4]+recruit_target_num:fl[7]].strip())
                # 참가비용ㄴ 우대사항ㅇ
                else:
                    a4 = re.sub(r"[-]","",test[fl[4]+recruit_target_num:fl[5]].strip())
                    a6 = re.sub(r"[-]","",test[fl[4]+recruit_target_num:fl[7]].strip())
            # 참가 비용 있다면
            else:
                # 참가비용 ㅇ 우대사항 ㄴ
                if preferential_treatment == -1:
                    a5 = re.sub(r"[-]","",test[fl[5]+9:fl[7]].strip())
                # 참가비용 ㅇ 우대사항 ㅇ
                else:
                    a5 = re.sub(r"[-]","",test[fl[5]+9:fl[6]].strip())
                    a6 = re.sub(r"[-]","",test[fl[6]+5:fl[7]].strip())

            a7 = re.sub(r"[-]","",test[fl[7]+5:fl[8]].strip())
            a8 = re.sub(r"[-]","",test[fl[8]+5:fl[9]].strip())

            if activity_content == -1:
                a1 = ""
            if recruit_date == -1:
                a2 = ""
            if recruit_people == -1:
                a3 = ""
            if recruit_target == -1:
                a4 = ""
            if fee == -1:
                a5 = ""
            if preferential_treatment == -1:
                a6 = ""
            if activity_nation == -1:
                a7 = ""
            if activity_bonus == -1:
                a8 = ""

            # 주최사 / 활동명에서 주최사만 분리
            if a0.find(',') is not -1:
                a0 = a0[ :a0.find(',')].strip()
            if a0.find('/') is not -1:
                a0 = a0[ :a0.find('/')].strip()

#             a1_sub = a1.find("(")
#             if a1_sub != -1:
#                 a1 = a1[:a1_sub]

        #         temp_list=[titles,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13]

            temp_list=[titles,dates,a0,a1,a2,a3,a4,a5,a6,a7,a8]
            try:
                f = open(name, 'a+', encoding='ansi',newline='')
                wr = csv.writer(f)
                wr.writerow(temp_list)
                f.close()
            except:
                pass

    return temp_list

-----

______

In [44]:
a = df["모집 기간"][0]
new_a = re.sub(r"[^0-9~]","",a)
fa = new_a.find('~')
first_a = new_a[:fa]
end_a = new_a[fa+1:]

if len(first_a) != 8:
    if first_a.find('2018') == -1:
        # ex) 7월 17일 => 717
        if len(first_a) == 3:
            if first_a[0] == '0':
                first_a += "2018"
            else:
                first_a += "20180"
        elif len(first_a) == 4:
            

NameError: name 'df' is not defined

In [200]:
a[:4]+'0'+a[4:]

'20180718'

In [45]:
if a.find('~'):
    new_a = re.sub(r"[^0-9~]","",a)

    af = new_a[:new_a.find('~')]
    al = new_a[new_a.find('~')+1:]

    if af[0] != 0:
        af = str(20180)+af

    if al[0] != 0:
        al = str(20180)+al

    print("start", af)
    print("end", al)

NameError: name 'a' is not defined

----